# Differential expression analysis


You have run the nf-core/rnaseq pipeline and checked the first quality control metrics of your fastq files. This was, however, only the primary analysis and we want to take it further.

Due to the computational demand of the pipeline, you only ran the pipeline on two of the 16 samples in the study yesterday. We provide you an essential output of nf-core/rnaseq pipeline in the `data` folder: It contains the combined epression matrix as produced by Salmon, which provides transcript levels for each gene (rows) and each sample (columns).


We would now like to understand exactly the difference between the expression in our groups of mice. 
Which pipeline would you use for this?

The differential abundance pipeline. 

Have a close look at the pipeline's "Usage" page on the [nf-core docs](nf-co.re). You will need to create a samplesheet (based on the column names in the provided matrix).

In [30]:
import pandas as pd
#create samplesheet
de_matrix = pd.read_csv("/Users/peterbrederlow/Documents/Uni/MasterBioinformatik/SS25/WorkflowCourse/computational-workflows-2025/notebooks/day_03/data/salmon.merged.gene_counts.tsv", sep = "\t",)
sample_names = de_matrix.columns[2:]
samplesheet = pd.DataFrame(sample_names, columns=["sample"])
# Extract conditions from the sample name
samplesheet["Group"] = samplesheet["sample"].apply(lambda x: x.split("_")[0])  # Sham / SNI
samplesheet["Genotype"] = samplesheet["sample"].apply(lambda x: x.split("_")[1])  # oxy / Sal
samplesheet["Condition"] = samplesheet["Group"] + "_" + samplesheet["Genotype"]
print(samplesheet.head())
samplesheet.to_csv("samplesheet.csv", index=False)

       sample Group Genotype Condition
0  Sham_oxy_1  Sham      oxy  Sham_oxy
1  Sham_oxy_2  Sham      oxy  Sham_oxy
2  Sham_oxy_3  Sham      oxy  Sham_oxy
3  Sham_oxy_4  Sham      oxy  Sham_oxy
4  Sham_Sal_1  Sham      Sal  Sham_Sal


In [31]:
duplicates = de_matrix.index[de_matrix.index.duplicated()]

duplicates = de_matrix['gene_name'][de_matrix['gene_name'].duplicated()]
print(duplicates)


15128           Mical3
30722             Nron
31110            Mtcp1
31286          Gm16042
31628          Gm15494
             ...      
45389            Srcap
45418         Arhgap30
45429           Pagr1a
45519    RP23-148J20.3
45666         Itgb1bp1
Name: gene_name, Length: 123, dtype: object


Please paste here the command you used. You may need to inspect the provided expression matrix more closely and create additional files, like a samplesheet (based on the column names) or a contrast file (there happens to also be one in `data/` that you can use).

In [32]:
nextflow run nf-core/differentialabundance -r 1.5.0\
    --input "/Users/peterbrederlow/Documents/Uni/MasterBioinformatik/SS25/WorkflowCourse/computational-workflows-2025/notebooks/day_03/samplesheet.csv"\
    --matrix "/Users/peterbrederlow/Documents/Uni/MasterBioinformatik/SS25/WorkflowCourse/computational-workflows-2025/notebooks/day_03/data/salmon.merged.gene_counts.tsv" \
    --contrasts "/Users/peterbrederlow/Documents/Uni/MasterBioinformatik/SS25/WorkflowCourse/computational-workflows-2025/notebooks/day_03/data/contrasts.csv" \
    -profile docker,arm \
    --outdir "differentialExpress" 

SyntaxError: invalid syntax (1107307534.py, line 1)

Explain all the parameters you set and why you set them in this way. If you used or created additional files as input, explain what they are used for.

The -profile flag contains the specification for the arm docker profile, additionally the input matrix of the counts for all Genes and samples wa specified, the samplesheet that describes the experimental condition and genotype, as well as the contrat.  
The contrast also specifies the conditions that should be compared to each other, and which group serves as baseline/reference. 

What were the outputs of the pipeline?

The output contains a report, which is base on a shinyapp, in which plots which can be separately looked at as well. Additionally, it contains a matrix with the normalized gene counts and matrices containing the differentially expressed genes with their regarding log fold change, p-value etc. 

In [2]:
#!TODO

Would you exclude any samples? If yes, which and why?

Based on the PCA, and also based on the median absolute deviation of the genes, the samples SNI_Sal 1 and 2 are probably outliers in their experimental group and should be excluded. 

In [34]:
#create new samplesheet
new_samplesheet = samplesheet
new_samplesheet = samplesheet.drop([13, 15], axis=0)

print(new_samplesheet)

new_samplesheet.to_csv("new_samplesheet.csv", index=False)

        sample Group Genotype Condition
0   Sham_oxy_1  Sham      oxy  Sham_oxy
1   Sham_oxy_2  Sham      oxy  Sham_oxy
2   Sham_oxy_3  Sham      oxy  Sham_oxy
3   Sham_oxy_4  Sham      oxy  Sham_oxy
4   Sham_Sal_1  Sham      Sal  Sham_Sal
5   Sham_Sal_2  Sham      Sal  Sham_Sal
6   Sham_Sal_3  Sham      Sal  Sham_Sal
7   Sham_Sal_4  Sham      Sal  Sham_Sal
8    SNI_oxy_1   SNI      oxy   SNI_oxy
9    SNI_oxy_2   SNI      oxy   SNI_oxy
10   SNI_oxy_3   SNI      oxy   SNI_oxy
11   SNI_oxy_4   SNI      oxy   SNI_oxy
12   SNI_Sal_1   SNI      Sal   SNI_Sal
14   SNI_Sal_3   SNI      Sal   SNI_Sal


In [ ]:
#the pipeline was rerun without the excluded samples
nextflow run nf-core/differentialabundance -r 1.5.0\
    --input "/Users/peterbrederlow/Documents/Uni/MasterBioinformatik/SS25/WorkflowCourse/computational-workflows-2025/notebooks/day_03/new_samplesheet.csv" \
    --matrix "/Users/peterbrederlow/Documents/Uni/MasterBioinformatik/SS25/WorkflowCourse/computational-workflows-2025/notebooks/day_03/data/salmon.merged.gene_counts.tsv" \
    --contrasts "/Users/peterbrederlow/Documents/Uni/MasterBioinformatik/SS25/WorkflowCourse/computational-workflows-2025/notebooks/day_03/data/contrasts.csv" \
    -profile docker,arm \
    --outdir "differentialExpress_filtered" 

How many genes were differentially expressed in each contrast? Does this confirm what the paper mentions?

Contrasts:  
Sal_oxy vs sal: 18  
Sham_oxy vs sal: 7  

The paper mentions differentially expressed genes in three brain regions : the NAc, mPFC and VTA. Briefly explain what these 3 regions are.

The ventral tegmental area (VTA) is a midbrain region where opioids disinhibit dopaminergic neurons, increasing dopamine release. This dopamine projects to the nucleus accumbens (NAc), driving reward, reinforcement, and drug-seeking behavior. The medial prefrontal cortex (mPFC) regulates decision-making and impulse control, but chronic opioid exposure can alter its activity, weakening behavioral control and promoting addiction.

Is there anyway from the paper and the material and methods for us to know which genes are included in these regions?

Once you have your list of differentially expressed genes, do you think just communicating those to the biologists would be sufficient? What does the publication state?

Please reproduce the Venn Diagram from Figure 3, not taking into account the brain regions but just the contrasts mentionned.